In [ ]:
# test script for complete get_nse_options

import pandas as pd
from ib_insync import *
util.startLoop()
from helper import get_connected, get_nse_options, get_nses

ib = get_connected('nse', 'live')

qlm = get_nses(ib)
undContracts = qlm[0]
lots_dict = qlm[1]
margins_dict = qlm[2]

tickers = ib.reqTickers(*undContracts)
undPrices = {t.contract.symbol: t.marketPrice() for t in tickers} # {symbol: undPrice}

u = undContracts[0]
p = list(undPrices.values())[0]
z = list(lots_dict.values())[0]
m = list(margins_dict.values())[0]

get_nse_options(ib, u, p, z, m)

ib.disconnect()

import pandas as pd
pd.read_pickle('../data/nse/ACC.pkl').head() 

In [ ]:
# test script for one symbol get_nse_option

from ib_insync import *
util.startLoop()
from helper import get_connected, get_nses
ib = get_connected('nse', 'live')

symbol = 'PNB'
lotsize = 7000
c = ib.qualifyContracts(Stock(symbol, 'NSE'))[0]
ib.reqSecDefOptParams(underlyingSymbol=symbol, futFopExchange='', underlyingSecType='STK', underlyingConId=c.conId)

strikes = (57.5, 60.0, 62.5)
os = [Option('PNB', '20190627', s, 'P', 'NSE') for s in strikes]
ocs = ib.qualifyContracts(*os)
order = Order(action='SELL', orderType='MKT', totalQuantity=lotsize, whatIf=True)

from helper import get_nses
qlm = get_nses(ib)

qlm[2]['PNB']

from itertools import repeat
{c.conId: ib.whatIfOrder(c, Order(action='SELL', orderType='MKT', \
                                 totalQuantity=abs(ls), whatIf=True)).initMarginChange
 for c, ls in (zip(ocs, repeat(lotsize)))}

ib.disconnect()

In [ ]:
# get_snp_remqty.py program test script
def get_snp_remqty(ib):
    '''generates the remaining quantities dictionary
    Args:
        (ib) as connection object
    Returns:
        remqty as a dictionary of {symbol: remqty}
        '''
    exchange = 'SMART'
    snp_assignment_limit = 50000
    lotsize=100
    
    # get the list of underlying contracts
    undContracts = get_snps(ib)
    c_dict = {u.symbol: u for u in undContracts} # {symbol: contract}
    
    tickers = ib.reqTickers(*undContracts)
    undPrices = {t.contract.symbol: t.marketPrice() for t in tickers} # {symbol: undPrice}

    df_und = \
        pd.DataFrame.from_dict(undPrices, orient='index', columns=['undPrice']).\
        join(pd.DataFrame.from_dict(c_dict, orient='index', columns=['undContract'])).dropna()
    
    df_und = df_und.assign(lotsize=lotsize)
    
    # remaining quantities in entire snp per assignment limit
    df_und = df_und.assign(remq=(snp_assignment_limit/(df_und.lotsize*df_und.undPrice)).astype('int')) 
    
    # from portfolio
    #_______________
    
    p = util.df(ib.portfolio()) # portfolio table
    
    # extract option contract info from portfolio table
    dfp = pd.concat([p, util.df([c for c in p.contract])[util.df([c for c in p.contract]).columns[:7]]], axis=1).iloc[:, 1:]
    dfp = dfp.rename(columns={'lastTradeDateOrContractMonth': 'expiration'})

    # get the underlying's margins, lots, prices and positions
    pos_dict = dfp.groupby('symbol')['position'].sum().to_dict()
    p_syms = {s for s in dfp.symbol}
    p_undc = {s: ib.qualifyContracts(Stock(s, exchange, currency='USD')) for s in p_syms}   # {symbol: contracts} dictionary
    undmlist = [get_margins(ib, u, 100) for u in p_undc.values()]                 # {contract: margin} dictionary
    p_undMargins = {k.symbol: v for i in undmlist for k, v in i.items()}        # {symbol: margin} dictionary
    p_undPrices = {u: undPrices[u] for u in p_syms}    #{symbol: undPrice} dictionary
    
    dfp1 = pd.DataFrame.from_dict(p_undc, orient='index', columns=['contract']). \
        join(pd.DataFrame.from_dict(p_undMargins, orient='index', columns=['undmargin'])). \
        join(pd.DataFrame.from_dict(p_undPrices, orient='index', columns=['undPrice'])). \
        join(pd.DataFrame.from_dict(pos_dict, orient='index', columns=['position']))
    
    dfp1 = dfp1.assign(undLot=1)
    
    dfp1 = dfp1.assign(qty=(dfp1.position/dfp1.undLot).astype('int'))
    
    # make the blacklist
    #___________________
    remqty_dict = pd.DataFrame(df_und.loc[dfp1.index].remq + dfp1.qty).to_dict()[0] # remq from portfolio
    remqty_dict = {k:(v if v > 0 else 0) for k, v in remqty_dict.items()} # portfolio's remq with negative values removed
    blacklist = [k for k, v in remqty_dict.items() if v <=0] # the blacklist
    df_und.remq.update(pd.Series(remqty_dict)) # replace all underlying with remq of portfolio
    remqty = df_und.remq.to_dict() # dictionary
    return remqty

In [ ]:
import pandas as pd
from ib_insync import *
util.startLoop()
from helper import get_connected, get_snps, get_margins
# ib = get_connected('snp', 'live')
remqty = get_snp_remqty(ib)
ib.disconnect()